# Chapter 26: Roles, Privileges, and Ownership

PostgreSQL's security model centers on roles—a unified concept encompassing users and groups—combined with a granular privilege system controlling access to database objects. Proper role architecture is foundational to security, separating application connections, administrative tasks, and read-only analytics while enforcing least privilege principles. This chapter establishes patterns for role design, privilege management, and object ownership hygiene in production environments.

## 26.1 Role Architecture Fundamentals

Roles are global cluster objects (visible across all databases) that own database objects and hold privileges. Understanding the distinction between login roles, group roles, and object ownership is essential for secure design.

### 26.1.1 Role Types and Attributes

```sql
-- View all roles in the cluster:
SELECT 
    rolname,
    rolsuper,
    rolinherit,
    rolcreaterole,
    rolcreatedb,
    rolcanlogin,
    rolreplication,
    rolbypassrls,
    rolconnlimit,
    rolvaliduntil
FROM pg_roles
WHERE rolname !~ '^pg_';  -- Exclude system roles

-- Key role attributes:
-- rolsuper: Superuser (bypasses all permission checks, use sparingly)
-- rolinherit: Inherit privileges from granted roles (default true, usually desired)
-- rolcanlogin: Can be used for connections (LOGIN privilege)
-- rolcreatedb: Can create databases
-- rolcreaterole: Can create other roles (not superusers unless rolsuper)
-- rolreplication: Can initiate streaming replication
-- rolbypassrls: Bypass Row-Level Security (usually only for table owners)
-- rolconnlimit: Maximum concurrent connections (-1 = unlimited)

-- Creating roles with specific attributes:

-- Application connection role (minimal privileges):
CREATE ROLE app_user WITH
    LOGIN
    PASSWORD 'strong_random_password'  -- Use generated passwords, rotate regularly
    NOSUPERUSER
    NOCREATEDB
    NOCREATEROLE
    INHERIT
    NOREPLICATION
    CONNECTION LIMIT 100;  -- Limit concurrent connections per role

-- Read-only reporting role:
CREATE ROLE readonly_user WITH
    LOGIN
    PASSWORD 'different_strong_password'
    NOSUPERUSER
    NOCREATEDB
    NOCREATEROLE
    INHERIT
    NOREPLICATION
    CONNECTION LIMIT 20;

-- Group role (no login, for privilege grouping):
CREATE ROLE data_analyst WITH
    NOLOGIN
    NOSUPERUSER
    NOCREATEDB
    NOCREATEROLE
    INHERIT;

-- Grant group membership:
GRANT data_analyst TO readonly_user;
-- Now readonly_user inherits all privileges granted to data_analyst
```

### 26.1.2 Role Inheritance and Membership

```sql
-- Role inheritance mechanics:
-- WITH INHERIT (default): Member automatically has privileges of granted role
-- WITHOUT INHERIT: Member must SET ROLE to assume privileges

-- Demonstration:
CREATE ROLE parent_role NOLOGIN;
CREATE ROLE child_with_inherit LOGIN PASSWORD 'test' INHERIT;
CREATE ROLE child_no_inherit LOGIN PASSWORD 'test' NOINHERIT;

GRANT parent_role TO child_with_inherit;
GRANT parent_role TO child_no_inherit;

-- Grant privilege to parent:
GRANT SELECT ON orders TO parent_role;

-- Test with_inherit:
-- psql -U child_with_inherit -d mydb
SELECT * FROM orders;  -- SUCCEEDS (inherits SELECT from parent_role)

-- Test no_inherit:
-- psql -U child_no_inherit -d mydb
SELECT * FROM orders;  -- FAILS: permission denied
SET ROLE parent_role;  -- Assume parent role explicitly
SELECT * FROM orders;  -- SUCCEEDS
RESET ROLE;  -- Return to original role

-- Practical use of NOINHERIT:
-- Admin role that shouldn't accidentally use elevated privileges
-- Must explicitly SET ROLE admin to perform admin tasks
-- Prevents privilege escalation bugs in application code

-- Group role best practices:
-- 1. Create functional group roles (app_read, app_write, app_admin)
-- 2. Grant object privileges to groups, not individual users
-- 3. Add login roles to appropriate groups
-- 4. Use NOINHERIT for highly privileged groups (defense in depth)
```

## 26.2 Privilege Management

Privileges control access to database objects. PostgreSQL's granular system allows precise control over who can read, modify, or administer specific objects.

### 26.2.1 Object Privileges (GRANT/REVOKE)

```sql
-- Available privileges by object type:

-- TABLES: SELECT, INSERT, UPDATE, DELETE, TRUNCATE, REFERENCES, TRIGGER
-- COLUMNS: SELECT (specific columns), UPDATE (specific columns)
-- SEQUENCES: USAGE, SELECT, UPDATE
-- DATABASES: CREATE, CONNECT, TEMPORARY, TEMP
-- SCHEMAS: CREATE, USAGE
-- FUNCTIONS: EXECUTE
-- LANGUAGES: USAGE
-- TABLESPACES: CREATE

-- Granting table privileges:
GRANT SELECT, INSERT, UPDATE ON orders TO app_user;
GRANT DELETE ON orders TO app_admin;  -- Only admin can delete

-- Column-level grants (finer granularity):
GRANT SELECT (order_id, customer_id, total, status) ON orders TO readonly_user;
-- readonly_user cannot see: payment_info, internal_notes columns

-- Granting sequence usage (required for SERIAL/IDENTITY):
GRANT USAGE, SELECT ON SEQUENCE orders_order_id_seq TO app_user;
-- Or for all sequences in schema:
GRANT USAGE ON ALL SEQUENCES IN SCHEMA public TO app_user;

-- Schema privileges:
GRANT USAGE ON SCHEMA public TO app_user;  -- Access objects in schema
GRANT CREATE ON SCHEMA staging TO app_admin;  -- Create objects in schema

-- Function execution:
GRANT EXECUTE ON FUNCTION calculate_tax(DECIMAL, DECIMAL) TO app_user;
GRANT EXECUTE ON ALL FUNCTIONS IN SCHEMA public TO app_user;

-- Database connection:
GRANT CONNECT ON DATABASE production TO app_user;
REVOKE CONNECT ON DATABASE production FROM PUBLIC;  -- Default deny

-- Revoking privileges:
REVOKE DELETE ON orders FROM app_user;  -- Remove specific privilege
REVOKE ALL ON orders FROM app_user;  -- Remove all privileges
REVOKE ALL PRIVILEGES ON ALL TABLES IN SCHEMA public FROM app_user;

-- Checking current privileges:
-- For current user:
SELECT * FROM information_schema.table_privileges 
WHERE grantee = CURRENT_USER;

-- For specific table:
SELECT 
    grantee,
    privilege_type,
    is_grantable
FROM information_schema.table_privileges
WHERE table_name = 'orders';

-- Using has_*_privilege functions:
SELECT 
    has_database_privilege('app_user', 'production', 'CONNECT'),
    has_schema_privilege('app_user', 'public', 'USAGE'),
    has_table_privilege('app_user', 'orders', 'SELECT'),
    has_column_privilege('app_user', 'orders', 'total', 'UPDATE');
```

### 26.2.2 Default Privileges

```sql
-- Objects created in the future automatically get these privileges:
-- Set default privileges for objects created by a specific role:

-- As app_admin (the role that will create objects):
ALTER DEFAULT PRIVILEGES IN SCHEMA public
GRANT SELECT, INSERT, UPDATE, DELETE ON TABLES TO app_user;

ALTER DEFAULT PRIVILEGES IN SCHEMA public
GRANT USAGE, SELECT ON SEQUENCES TO app_user;

ALTER DEFAULT PRIVILEGES IN SCHEMA public
GRANT EXECUTE ON FUNCTIONS TO app_user;

-- Now any tables app_admin creates in public schema will automatically
-- have those privileges granted to app_user

-- View current default privileges:
SELECT 
    n.nspname as schema,
    defaclrole::regrole as granted_by,
    defaclacl::text as privileges
FROM pg_default_acl d
JOIN pg_namespace n ON d.defaclnamespace = n.oid;

-- Remove default privileges:
ALTER DEFAULT PRIVILEGES IN SCHEMA public
REVOKE ALL ON TABLES FROM app_user;

-- For roles that create objects (schema owners):
-- Ensure they set appropriate default privileges so application roles can access
-- new tables without manual GRANT statements
```

## 26.3 Object Ownership

Ownership determines who can modify object definitions and who bypasses RLS policies.

### 26.3.1 Ownership Transfer

```sql
-- Object owner has implicit full privileges and bypasses RLS
-- Change ownership with ALTER:

-- Table ownership:
ALTER TABLE orders OWNER TO app_admin;

-- Schema ownership:
ALTER SCHEMA public OWNER TO app_admin;

-- Database ownership:
ALTER DATABASE production OWNER TO app_admin;

-- Sequence ownership (automatic with table usually):
ALTER SEQUENCE orders_order_id_seq OWNER TO app_admin;

-- Function ownership:
ALTER FUNCTION calculate_tax(DECIMAL, DECIMAL) OWNER TO app_admin;

-- Bulk ownership change:
DO $$
DECLARE
    r RECORD;
BEGIN
    FOR r IN 
        SELECT tablename 
        FROM pg_tables 
        WHERE schemaname = 'public' 
          AND tableowner = 'old_owner'
    LOOP
        EXECUTE format('ALTER TABLE %I OWNER TO new_owner', r.tablename);
    END LOOP;
END $$;

-- Ownership and RLS:
-- Table owner bypasses RLS policies (can see all rows)
-- This is why application should not connect as table owner
-- Use separate roles: app_owner (DDL), app_user (DML, RLS-respecting)

-- Checking current ownership:
SELECT 
    n.nspname as schema,
    c.relname as object,
    c.relkind as type,
    pg_get_userbyid(c.relowner) as owner
FROM pg_class c
JOIN pg_namespace n ON c.relnamespace = n.oid
WHERE n.nspname = 'public'
  AND c.relkind IN ('r', 'v', 'S', 'f');
```

### 26.3.2 Schema Hygiene

```sql
-- Separate schemas by purpose and ownership:

-- 1. Application schema (app tables):
CREATE SCHEMA app_data;
ALTER SCHEMA app_data OWNER TO app_owner;
GRANT USAGE ON SCHEMA app_data TO app_user;
GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA app_data TO app_user;

-- 2. Extensions schema (isolated):
CREATE SCHEMA extensions;
CREATE EXTENSION pgcrypto WITH SCHEMA extensions;
REVOKE ALL ON SCHEMA extensions FROM PUBLIC;  -- Hide implementation details
GRANT USAGE ON SCHEMA extensions TO app_user;  -- But allow function execution

-- 3. Audit schema (restricted):
CREATE SCHEMA audit;
CREATE TABLE audit.log_entries (...);
ALTER SCHEMA audit OWNER TO audit_admin;
GRANT INSERT ON audit.log_entries TO app_user;  -- Can only insert, not read
-- Audit triggers run as owner (SECURITY DEFINER) to write to restricted schema

-- 4. Migration/schema change schema (temporary):
CREATE SCHEMA migration_2024_01;
-- Apply changes here first, test, then move to app_data
-- Allows rollback by dropping schema

-- Schema search path security:
-- Never put untrusted schemas early in search_path
-- Set at database level:
ALTER DATABASE production SET search_path = app_data, extensions, public;
-- Prevent public schema from being used for object creation:
REVOKE CREATE ON SCHEMA public FROM PUBLIC;

-- Object visibility:
-- Users can only see objects in schemas they have USAGE privilege on
-- This provides logical isolation even without RLS
```

## 26.4 Least Privilege Checklist

Implementing least privilege requires systematic restriction of default permissions and explicit grants for necessary access.

### 26.4.1 Database Hardening Steps

```sql
-- Step 1: Remove public schema privileges
REVOKE ALL ON SCHEMA public FROM PUBLIC;
REVOKE CREATE ON SCHEMA public FROM PUBLIC;

-- Step 2: Create application-specific roles
CREATE ROLE app_read NOLOGIN;
CREATE ROLE app_write NOLOGIN;
CREATE ROLE app_admin NOLOGIN;

-- Step 3: Grant privileges to group roles
GRANT USAGE ON SCHEMA app_data TO app_read, app_write, app_admin;
GRANT SELECT ON ALL TABLES IN SCHEMA app_data TO app_read;
GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA app_data TO app_write;
GRANT ALL PRIVILEGES ON ALL TABLES IN SCHEMA app_data TO app_admin;
GRANT USAGE ON ALL SEQUENCES IN SCHEMA app_data TO app_write, app_admin;

-- Step 4: Create login roles that inherit from groups
CREATE ROLE app_user_prod WITH LOGIN PASSWORD '...' IN ROLE app_write;
CREATE ROLE app_user_read WITH LOGIN PASSWORD '...' IN ROLE app_read;

-- Step 5: Set default privileges for future objects
ALTER DEFAULT PRIVILEGES IN SCHEMA app_data
GRANT SELECT ON TABLES TO app_read;

ALTER DEFAULT PRIVILEGES IN SCHEMA app_data
GRANT SELECT, INSERT, UPDATE, DELETE ON TABLES TO app_write;

-- Step 6: Restrict superuser access
-- Ensure application never connects as postgres or other superuser
-- Create separate admin roles for specific tasks

-- Step 7: Audit privilege usage
-- Enable pgaudit extension if available to track privilege escalation
```

### 26.4.2 Privilege Verification Queries

```sql
-- Verify effective permissions for a role:
SELECT 
    table_schema,
    table_name,
    privilege_type
FROM information_schema.table_privileges
WHERE grantee = 'app_user_prod'
ORDER BY table_schema, table_name;

-- Check for privilege escalation paths:
-- Roles that can create other roles or databases
SELECT 
    rolname,
    rolcreaterole,
    rolcreatedb,
    rolsuper
FROM pg_roles
WHERE rolcreaterole OR rolcreatedb OR rolsuper;

-- Find objects owned by roles that shouldn't own them:
SELECT 
    n.nspname as schema,
    c.relname as object,
    c.relkind as type,
    pg_get_userbyid(c.relowner) as owner
FROM pg_class c
JOIN pg_namespace n ON c.relnamespace = n.oid
WHERE c.relowner != (SELECT oid FROM pg_roles WHERE rolname = 'app_owner')
  AND n.nspname = 'app_data'
  AND c.relkind IN ('r', 'v', 'S');

-- Verify default privileges are set correctly:
SELECT 
    n.nspname as schema,
    pg_get_userbyid(defaclrole) as role,
    defaclobjtype as object_type,
    defaclacl as privileges
FROM pg_default_acl d
JOIN pg_namespace n ON d.defaclnamespace = n.oid;
```

---

## Chapter Summary

In this chapter, you learned:

1. **Role Hierarchy**: PostgreSQL unifies users and groups into roles with attributes (`LOGIN`, `SUPERUSER`, `CREATEDB`, `CREATEROLE`). Group roles (without `LOGIN`) aggregate privileges assigned to member roles via `GRANT group_role TO login_role`. Role inheritance (`INHERIT` attribute) determines whether member roles automatically possess group privileges or must explicitly `SET ROLE`.

2. **Privilege Granularity**: Access control operates at multiple levels—database (`CONNECT`), schema (`USAGE`, `CREATE`), table (`SELECT`, `INSERT`, `UPDATE`, `DELETE`, `TRUNCATE`, `REFERENCES`, `TRIGGER`), column (`SELECT`/`UPDATE` specific columns), and sequence (`USAGE`, `SELECT`, `UPDATE`). Default privileges (`ALTER DEFAULT PRIVILEGES`) automatically apply to objects created in the future, ensuring consistent access control without manual grants.

3. **Object Ownership**: The role that creates an object owns it, gaining implicit full privileges and bypassing Row-Level Security. Ownership transfer (`ALTER ... OWNER TO`) requires membership in both old and new owner roles (or superuser). Application roles should never own tables (use dedicated owner roles for DDL, separate read/write roles for DML).

4. **Schema Hygiene**: Isolate extensions in dedicated schemas (`extensions`) with restricted `USAGE` grants. Remove `CREATE` privileges from `public` schema to prevent unauthorized object creation. Application data resides in dedicated schemas (`app_data`) with explicit grants, preventing privilege escalation through search_path manipulation.

5. **Least Privilege Implementation**: Systematically revoke default permissions (`REVOKE ALL ON SCHEMA public FROM PUBLIC`) and explicitly grant only necessary access. Use group roles (`app_read`, `app_write`, `app_admin`) to aggregate permissions, assigning login roles to appropriate groups. Regularly audit effective permissions via `information_schema.table_privileges` and `pg_roles` to detect privilege creep.

6. **Extension Security**: Trusted extensions operate within database security boundaries; untrusted extensions (requiring `shared_preload_libraries` or C libraries) demand superuser installation and careful access control. Never install untrusted extensions in production without security review. Wrap untrusted functionality in `SECURITY DEFINER` functions with safe `search_path` settings when application access is required.

**Next:** In Chapter 27, we will explore Authentication, pg_hba.conf, and TLS—covering authentication methods, host-based access control configuration, certificate management, and secure connection string patterns for production deployments.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../6. Programmability/25. extensions_and_ecosystem.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='27. authentication_pg_hbaconf_and_tls.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
